In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from sklearn import metrics
import lightgbm as lgbm
import numpy as np

In [2]:
train = pd.read_csv("../input/nlp-getting-started/train.csv")
test = pd.read_csv("../input/nlp-getting-started/test.csv")

In [3]:
##Step2: data cleaning
pattern = '[^a-z]'
stopwords = set(stopwords.words('english'))

def clean_data(text):
    
    # to lowercase
    text = text.lower()

    #remove URL
    text =  re.sub('http[s]?://\S+', '', text)

    # remove non-letters
    text = re.sub(pattern, ' ', text)

    # remove whitespaces
    text = " ".join(text.split())

    #tokenizer
    text = word_tokenize(text)

    #remove stopwords
    output= ' '.join([i for i in text if i not in stopwords])

    return output

In [4]:
##Step3: Split in to Training and Test Data
x = train['text']
y = train["target"]

clean_messages = []
for message in x:
    clean_messages.append(clean_data(message))
    
x = pd.DataFrame(clean_messages, columns=['Text'])

In [5]:
freq_word = x["Text"].str.split(expand=True).stack().value_counts().loc[lambda x: x<5].reset_index()
freq_word.columns = ["words", "frequency"]

def word_freq(text, removeWord):

    text = word_tokenize(text)

    output_text = " ".join(i for i in text if i not in removeWord["words"].values)

    return output_text

In [6]:
clean_tweet = []
x = x['Text']

for message in x:
    clean_tweet.append(word_freq(message, freq_word))
X = pd.DataFrame(clean_tweet, columns=['Text'])

In [7]:
cv = CountVectorizer(dtype=np.float32)
features = cv.fit_transform(X['Text'])


##Step5: Build a model svm
model = svm.SVC(C = 1000, gamma = 0.0001, kernel = 'rbf', random_state = 1) 
model.fit(features,y)

##step7 confusion matrix
pred = model.predict(cv.transform(test["text"]))
#print(metrics.confusion_matrix(y_test, pred))
#print(metrics.classification_report(y_test, pred))

output = pd.DataFrame({'ID': test["id"], 'target': pred})
output.to_csv('submission.csv', index=False)

#rf = lgbm.LGBMClassifier(random_state = 1)
#rf.fit(features,y)
#result = rf.predict(cv.transform(test["text"]))

#output = pd.DataFrame({'ID': test["id"], 'target': result})
#output.to_csv('submission1.csv', index=False)